In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")


In [3]:
from langchain_groq import ChatGroq

model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Shubham and I am a chief ai engineer")])

AIMessage(content="Hello Shubham,\n\nIt's nice to meet you! Thanks for introducing yourself. As a chief AI engineer, what kind of exciting projects are you working on?  \n\nI'm always eager to learn more about the innovative applications of AI.  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 23, 'total_tokens': 79, 'completion_time': 0.101818182, 'prompt_time': 0.001329369, 'queue_time': 0.24999522700000001, 'total_time': 0.103147551}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--3f9ec831-b6f5-4694-b566-6c14ffeb94b2-0', usage_metadata={'input_tokens': 23, 'output_tokens': 56, 'total_tokens': 79})

In [5]:
from langchain_core.messages import AIMessage
from sympy import content
model.invoke(
    [
        HumanMessage(content="Hi, My name is Shubham and I am a chief ai engineer"),
        AIMessage(content="Hello Shubham, it's nice to meet you!\n\nThat's a fascinating title. As a large language model, I'm always interested in learning more about how people are using AI in innovative ways. \n\nWhat kind of projects are you working on as Chief AI Engineer?  \n\n"),
        HumanMessage(content="Hey, What's my name and what do i do?")
    ]
)

AIMessage(content="You told me your name is Shubham and that you are a Chief AI Engineer!  \n\nIs there anything else you'd like to tell me about yourself or your work? I'm eager to learn more. 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 107, 'total_tokens': 156, 'completion_time': 0.089090909, 'prompt_time': 0.00342906, 'queue_time': 0.25021428100000004, 'total_time': 0.092519969}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--afba6f32-a3c4-472a-9cc2-3705a7d11e5b-0', usage_metadata={'input_tokens': 107, 'output_tokens': 49, 'total_tokens': 156})

In [6]:
## Message history

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [7]:
config = {"configurable":{"session_id":"chat1"}}

In [8]:
response = with_message_history.invoke(
    [HumanMessage(content = "Hi, My name is Shubham and I am a chief ai engineer")],
    config=config
)

In [9]:
response.content

"Hello Shubham,\n\nIt's nice to meet you! Being a Chief AI Engineer is a fascinating role. What kind of projects are you currently working on?\n\nI'm always eager to learn more about the cutting-edge applications of AI. Perhaps you could tell me about some of the challenges and rewards you encounter in your work.\n"

In [10]:
response = with_message_history.invoke(
    [HumanMessage(content = "Hi, My name is Shubham and I am a chief ai engineer")],
    config=config
)

In [11]:
with_message_history.invoke(
    [HumanMessage(content = "What is my name?")],
    config=config
)

AIMessage(content='Your name is Shubham.  😊  \n\nI remember that from our first conversation!  \n\n\n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 194, 'total_tokens': 217, 'completion_time': 0.041818182, 'prompt_time': 0.004872908, 'queue_time': 0.249054618, 'total_time': 0.04669109}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--39f8868a-bf49-47ba-99fc-d35bbcc070fd-0', usage_metadata={'input_tokens': 194, 'output_tokens': 23, 'total_tokens': 217})

In [12]:
## changing the config-->session id
config1 = {"configurable":{"session_id":"chat2"}}

In [13]:
with_message_history.invoke(
    [HumanMessage(content = "My name is Shubham")],
    config=config1
)

AIMessage(content="Hello Shubham!\n\nIt's nice to meet you. 👋\n\nIs there anything I can help you with today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 14, 'total_tokens': 42, 'completion_time': 0.050909091, 'prompt_time': 0.00125414, 'queue_time': 0.24971352, 'total_time': 0.052163231}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--bcde5f3f-7ad3-4115-8a4a-ae1a1585debf-0', usage_metadata={'input_tokens': 14, 'output_tokens': 28, 'total_tokens': 42})

In [14]:
print(config)
print(get_session_history("chat2"))

{'configurable': {'session_id': 'chat1'}}
Human: My name is Shubham
AI: Hello Shubham!

It's nice to meet you. 👋

Is there anything I can help you with today?



In [15]:
## Prompt templates

from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the question to the best of your ability. You should be very friendly and anyone should love to talk with you."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model


In [16]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is Shubham")]})

AIMessage(content="Hi Shubham, it's so nice to meet you! 👋  I'm happy to be your helpful assistant. What can I do for you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 46, 'total_tokens': 85, 'completion_time': 0.070909091, 'prompt_time': 0.00162954, 'queue_time': 0.252227976, 'total_time': 0.072538631}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--7abaf18c-14b3-4ff9-a998-339ee56f3c64-0', usage_metadata={'input_tokens': 46, 'output_tokens': 39, 'total_tokens': 85})

In [17]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [18]:
config = {"configurable":{"session_id":"chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi my name is Shubham")],
    config = config
)

response

AIMessage(content="Hi Shubham! It's so nice to meet you! 👋  \n\nWhat can I do for you today? I'm happy to answer any questions you have or just chat about whatever's on your mind. 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 46, 'total_tokens': 97, 'completion_time': 0.092727273, 'prompt_time': 0.001705429, 'queue_time': 0.24976553199999998, 'total_time': 0.094432702}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--f3652ecc-cfe6-4077-9d40-8c822d49ca63-0', usage_metadata={'input_tokens': 46, 'output_tokens': 51, 'total_tokens': 97})

In [19]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the question to the best of your ability in this {language}. You should be very friendly and anyone should love to talk with you."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [20]:
response = chain.invoke({"messages":[HumanMessage(content="Hi my name is Shubham")],"language":"Hindi"})
response.content

'नमस्ते Shubham! 😊  \n\nबहुत ही अच्छा है तुम्हें यहाँ देखकर!  मुझे बहुत खुशी है कि तुम मेरे साथ बात करना चाह रहे हो। \n\nक्या मैं तुम्हारे लिए कुछ कर सकता हूँ?  😜\n\n'

In [21]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [22]:
config = {"configurable":{"session_id":"chat4"}}
response = with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi my name is Shubham")],"language":"Hindi"},
    config = config
)

In [23]:
response.content

'नमस्ते Shubham! 😊 \n\nबहुत अच्छा है तुम्हें यहाँ देखकर। क्या मैं तुम्हें कुछ मदद कर सकता हूँ?  \n \nशुक्रिया! आपसे बात करना बहुत अच्छा लग रहा है। 😊  \n\n'

In [24]:
response = with_message_history.invoke(
    {'messages': [HumanMessage(content="Whats my name?")],"language":"Hindi"},
    config = config
)
response.content

'आपका नाम Shubham है! 😊 \n\n\n'

### Managing conversation history


In [27]:
## Managing conversation history

from langchain_core.messages import SystemMessage,trim_messages

trimmer = trim_messages(
    max_tokens = 70,
    strategy="last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on="human"
)

messages = [

    SystemMessage(content="You're a good assistant"),
    HumanMessage(content="Hi! I'm Bob"),
    AIMessage(content="Hi Bob!"),
    HumanMessage(content="Can you help me write a short email?"),
    AIMessage(content="Sure, go ahead."),
    HumanMessage(content="I want to thank my manager for the promotion."),
    AIMessage(content="Nice! Congratulations."),
    HumanMessage(content="Can you draft it for me?"),
    AIMessage(content="Yes, one moment."),
    HumanMessage(content="Also, make it sound professional."),
    AIMessage(content="Got it."),
    HumanMessage(content="Thanks!"),
    AIMessage(content="Anytime!"),

]

trimmer.invoke(messages)

d:\AI Udemy Course\Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\AI Udemy Course\Langchain\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\asus\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this ar

[SystemMessage(content="You're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I want to thank my manager for the promotion.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice! Congratulations.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Can you draft it for me?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes, one moment.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Also, make it sound professional.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Got it.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Anytime!', additional_kwargs={}, response_metadata={})]

In [32]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
    "messages": messages + [HumanMessage(content="What task i gave to you?")],
    "language":"English"
    }
)

In [33]:
response.content

"You asked me to draft something for you, and to make it sound professional.  \n\nIs there anything else I can help you with? Perhaps you could tell me what you'd like me to draft so I can get started! 😊  \n"

In [34]:
## Lets wrap this in the message history

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

config ={"configurable":{"session_id":"chat5"}}

In [41]:
response = with_message_history.invoke(
    {
    "messages": [HumanMessage(content="My name is Shubham, remember my name")],
    "language":"English"
    },
    config=config
)

response.content

"It's lovely to meet you, Shubham!  I'll remember your name.  What can I do for you today? 😊  \n"

In [42]:
with_message_history.invoke(
    {
    "messages": [HumanMessage(content="What is my name?")],
    "language":"English"
    },
    config=config
)

AIMessage(content='Your name is Shubham!  Nice to meet you, Shubham!  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 105, 'total_tokens': 135, 'completion_time': 0.054545455, 'prompt_time': 0.00274399, 'queue_time': 0.25286348000000003, 'total_time': 0.057289445}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2c299ccd-331f-41ab-becc-a75d20545ea3-0', usage_metadata={'input_tokens': 105, 'output_tokens': 30, 'total_tokens': 135})

In [43]:
get_session_history("chat5")

InMemoryChatMessageHistory(messages=[SystemMessage(content="You're a good assistant", additional_kwargs={}, response_metadata={}), HumanMessage(content="Hi! I'm Bob", additional_kwargs={}, response_metadata={}), AIMessage(content='Hi Bob!', additional_kwargs={}, response_metadata={}), HumanMessage(content='Can you help me write a short email?', additional_kwargs={}, response_metadata={}), AIMessage(content='Sure, go ahead.', additional_kwargs={}, response_metadata={}), HumanMessage(content='I want to thank my manager for the promotion.', additional_kwargs={}, response_metadata={}), AIMessage(content='Nice! Congratulations.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Can you draft it for me?', additional_kwargs={}, response_metadata={}), AIMessage(content='Yes, one moment.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Also, make it sound professional.', additional_kwargs={}, response_metadata={}), AIMessage(content='Got it.', additional_kw